# DTCNN-ELM for LIDC-IDRI

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import datetime
import time
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from utils import make_dir
import hpelm
from sklearn import svm
from models import efficientnetb5_model
from models import mobilenetv2_model
from models import nasnetmobile_model
from models import resnet50_model
from models import xception_model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import math
import os
import numpy as np
from keras.models import Model
from keras.utils import to_categorical
import keras
import warnings
warnings.filterwarnings("ignore")


# learning rate

def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# Config


class DefaultConfigs(object):
    # --------------------------------------- Parameters --------------------------------------
    # String parameters
    PROJECT = 'LIDC'
    DTCNN_NAME = {0: 'ResNet50',
                  1: 'Xception',
                  2: 'EfficientNet-B5',
                  3: 'NASNetMobile',
                  4: 'MobileNetV2'}
    CLASSES_NAME = {0: 'Benign', 1: 'Malignant'}
    # Numeric parameters
    SEED = 66  # seed
    EP = 10  # epochs
    BS = 64  # batch_size
    NUM_CLASSES = 2
    LR = 2e-5  # fixed learning rate (test)
    DATE_STR = datetime.datetime.now().strftime('%Y%m%d')
    # -----------------------------------------------------------------------------------------

    # --------------------------------------- Directory ---------------------------------------
    # Data directory
    DATA_PATH = "data/demo.npz"
    #DATA_PATH = "data/LIDC-IDRI.npz"
    # Save directory
    CHECKPOINTS_SAVE_DIR = "./results/checkpoints/"
    BEST_MODEL_SAVE_DIRbest = CHECKPOINTS_SAVE_DIR + "best_model/"
    RESUTLS_SAVE_DIR = "./results/"
    LOGS_SAVE_DIR = RESUTLS_SAVE_DIR + "logs/"
    # -----------------------------------------------------------------------------------------


conf = DefaultConfigs()
np.random.seed(conf.SEED)
scaler = StandardScaler()

In [ ]:
datasets = np.load(conf.DATA_PATH)
datas_raw = datasets["imgs"]
labels = datasets["labels"]
datas_reshape = datas_raw.reshape(datas_raw.shape[0],64,64)  
datas = datas_reshape.astype('float32')
datas /= 255.
print(datas.shape)
print(labels.shape)

lrate = LearningRateScheduler(step_decay)

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=conf.SEED)
Acc1 = []
Sen1 = []
Spc1 = []
AUC1 = []
Train_time1 = []
Test_time1 = []

Acc2 = []
Sen2 = []
Spc2 = []
AUC2 = []
Train_time2 = []
Test_time2 = []

Acc3 = []
Sen3 = []
Spc3 = []
AUC3 = []
Train_time3 = []
Test_time3 = []

iLayer = 1
iFold = 1

for train, test in kfold.split(datas, labels):
    print('-----------Fold %s Begin-----------' % iFold)
    train_data = datas[train]
    train_label = to_categorical(labels[train], 2)
    test_data = datas[test]
    test_label = to_categorical(labels[test], 2)
    
    ##################
    ## DTCNN Softmax
    ##################
    # Create the source DCNN model
    model = xception_model((64,64),conf.NUM_CLASSES) 
    # Target DTCNN model with deep transfer learning
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train target DTCNN with Softmax classifier
    dtcnn_softmax_train_start = time.time()
    
    model_hst = model.fit(x=train_data, y=train_label,
                          batch_size=conf.BS, epochs=conf.EP,
                          callbacks=[lrate],
                          verbose=0,
                          )
    
    dtcnn_softmax_train_end = time.time()
    
    # Test Target DTCNN with Softmax classifier
    dtcnn_softmax_test_sart = time.time()
    
    dtcnn_pre = model.predict(test_data)
    
    dtcnn_softmax_test_end = time.time()
    
    # Evaluation for DTCNN with softmax
    dtcnn_class_results = np.array([np.argmax(r) for r in dtcnn_pre])
    tn1, fp1, fn1, tp1 = confusion_matrix(labels[test], dtcnn_class_results).ravel()
    accuracy1 = (tp1+tn1)/(tp1+tn1+fp1+fn1)
    sensitivity1 = tp1/(tp1+fn1)
    specificity1 = tn1/(tn1+fp1)
    auc1 = roc_auc_score(labels[test], dtcnn_class_results)
    dtcnn_softmax_train_time = dtcnn_softmax_train_end - dtcnn_softmax_train_start
    dtcnn_softmax_test_time = dtcnn_softmax_test_end - dtcnn_softmax_test_sart
    print('dtcnn_softmax_acc: %.4f' % accuracy1)
    print('dtcnn_softmax_sen: %.4f' % sensitivity1)
    print('dtcnn_softmax_spc: %.4f' % specificity1)
    print('dtcnn_softmax_auc: %.4f' % auc1)
    print('dtcnn_softmax_training_time: %.4f s' % dtcnn_softmax_train_time)
    print('dtcnn_softmax_testing_time: %.4f s' % dtcnn_softmax_test_time)
    Acc1.append(accuracy1 * 100)
    Sen1.append(sensitivity1 * 100)
    Spc1.append(specificity1 * 100)
    AUC1.append(auc1 * 100)
    Train_time1.append(dtcnn_softmax_train_time)
    Test_time1.append(dtcnn_softmax_test_time)
    
    ##################
    ## DTCNN ELM
    ##################
    layer_name = 'global_average_pooling2d_' + str(iLayer)
    iLayer = iLayer + 1
    dtcnn_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    
    # train DTCNN-ELM (two step)
    dtcnn_elm_train_strat1 = time.time()
    dtcnn_elm_train_feature = dtcnn_model.predict(train_data)
    dtcnn_elm_train_end1 = time.time()
    
    elm_model = hpelm.hp_elm.HPELM(dtcnn_elm_train_feature.shape[1], 2)
    elm_model.add_neurons(500, func='sigm')
    
    dtcnn_elm_train_strat2 = time.time()
    elm_model.train(dtcnn_elm_train_feature, train_label, 'c')
    dtcnn_elm_train_end2 = time.time()
    
    # test DTCNN-ELM 
    dtcnn_elm_test_strat = time.time()
    dtcnn_elm_test_feature = dtcnn_model.predict(test_data)
    dtcnn_elm_test = elm_model.predict(dtcnn_elm_test_feature)
    dtcnn_elm_test_end = time.time()
    
    # Evaluation for DTCNN-ELM
    dtcnn_elm_class_results = np.array([np.argmax(r) for r in dtcnn_elm_test])
    tn2, fp2, fn2, tp2 = confusion_matrix(labels[test], dtcnn_elm_class_results).ravel()
    accuracy2 = (tp2+tn2)/(tp2+tn2+fp2+fn2)
    sensitivity2 = tp2/(tp2+fn2)
    specificity2 = tn2/(tn2+fp2)
    auc2 = roc_auc_score(labels[test], dtcnn_elm_class_results)
    dtcnn_elm_train_time = dtcnn_elm_train_end1-dtcnn_elm_train_strat1+dtcnn_elm_train_end2-dtcnn_elm_train_strat2
    dtcnn_elm_test_time = dtcnn_elm_test_end - dtcnn_elm_test_strat
    print('dtcnn_elm_acc: %.4f' % accuracy2)
    print('dtcnn_elm_sen: %.4f' % sensitivity2)
    print('dtcnn_elm_spc: %.4f' % specificity2)
    print('dtcnn_elm_auc: %.4f' % auc2)
    print('dtcnn_elm_training_time: %.4f s' % dtcnn_elm_train_time)
    print('dtcnn_elm_testing_time: %.4f s' % dtcnn_elm_test_time)

    Acc2.append(accuracy2 * 100)
    Sen2.append(sensitivity2 * 100)
    Spc2.append(specificity2 * 100)
    AUC2.append(auc2 * 100)
    Train_time2.append(dtcnn_elm_train_time)
    Test_time2.append(dtcnn_elm_test_time)
    
    ##################
    ## DTCNN SVM
    ##################
    svm_model = svm.SVC(C=0.5)
    #c_range = np.array([0.0005, 0.005, 0.05, 0.5, 5])
    #param_grid = [{'kernel': ['rbf'], 'C': c_range}]
    #grid = GridSearchCV(svm_model, param_grid, n_jobs=-1, cv=kfold)
    
    # train DTCNN-SVM
    dtcnn_svm_train_strat = time.time()
    dtcnn_svm_train_feature = dtcnn_model.predict(train_data)
    #dtcnn_svm_train_feature_std = scaler.fit_transform(dtcnn_svm_train_feature)
    svm_model.fit(dtcnn_svm_train_feature, labels[train])
    #svm_grid = grid.fit(dtcnn_svm_train_feature_std, labels[train])
    #print("Best: %f using %s" % (svm_grid.best_score_,grid.best_params_))
    dtcnn_svm_train_end = time.time()
    
    # test DTCNN-SVM
    dtcnn_svm_test_strat = time.time()
    dtcnn_svm_test_feature = dtcnn_model.predict(test_data)
    #dtcnn_svm_test_feature_std = scaler.fit_transform(dtcnn_svm_test_feature)
    dtcnn_svm_result = svm_model.predict(dtcnn_svm_test_feature)
    dtcnn_svm_test_end = time.time()
    
    # Evaluation for DTCNN-SVM
    tn3, fp3, fn3, tp3 = confusion_matrix(labels[test], dtcnn_svm_result).ravel()
    accuracy3 = (tp3+tn3)/(tp3+tn3+fp3+fn3)
    sensitivity3 = tp3/(tp3+fn3)
    specificity3 = tn3/(tn3+fp3)
    auc3 = roc_auc_score(labels[test], dtcnn_svm_result)
    dtcnn_svm_train_time = dtcnn_svm_train_end - dtcnn_svm_train_strat
    dtcnn_svm_test_time = dtcnn_svm_test_end - dtcnn_svm_test_strat
    print('dtcnn_svm_acc: %.4f' % accuracy3)
    print('dtcnn_svm_sen: %.4f' % sensitivity3)
    print('dtcnn_svm_spc: %.4f' % specificity3)
    print('dtcnn_svm_auc: %.4f' % auc3)
    print('dtcnn_svm_training_time: %.4f s' % dtcnn_svm_train_time)
    print('dtcnn_svm_testing_time: %.4f s' % dtcnn_svm_test_time)

    Acc3.append(accuracy3 * 100)
    Sen3.append(sensitivity3 * 100)
    Spc3.append(specificity3 * 100)
    AUC3.append(auc3 * 100)
    Train_time3.append(dtcnn_svm_train_time)
    Test_time3.append(dtcnn_svm_test_time)
    
    print('-----------Fold %s End-----------' % iFold)
    iFold = iFold + 1
    
# Mean results
print("-----------DTCNN_Softmax-----------")
print("Acc: %.2f%% (+/- %.2f%%)" % (np.mean(Acc1), np.std(Acc1)))
print("Sen: %.2f%% (+/- %.2f%%)" % (np.mean(Sen1), np.std(Sen1)))
print("Spc: %.2f%% (+/- %.2f%%)" % (np.mean(Spc1), np.std(Spc1)))
print("AUC: %.2f%% (+/- %.2f%%)" % (np.mean(AUC1), np.std(AUC1)))
print("Training Time: %.4f s" % (np.mean(Train_time1)))
print("Testing Time: %.4f s" % (np.mean(Test_time1)))
print("-----------DTCNN_Softmax-----------")
print("-----------------------------------")
print("-------------DTCNN_ELM-------------")
print("Acc: %.2f%% (+/- %.2f%%)" % (np.mean(Acc2), np.std(Acc2)))
print("Sen: %.2f%% (+/- %.2f%%)" % (np.mean(Sen2), np.std(Sen2)))
print("Spc: %.2f%% (+/- %.2f%%)" % (np.mean(Spc2), np.std(Spc2)))
print("AUC: %.2f%% (+/- %.2f%%)" % (np.mean(AUC2), np.std(AUC2)))
print("Training Time: %.4f s" % (np.mean(Train_time2)))
print("Testing Time: %.4f s" % (np.mean(Test_time2)))
print("-------------DTCNN_ELM-------------")
print("-----------------------------------")
print("-------------DTCNN_SVM-------------")
print("Acc: %.2f%% (+/- %.2f%%)" % (np.mean(Acc3), np.std(Acc3)))
print("Sen: %.2f%% (+/- %.2f%%)" % (np.mean(Sen3), np.std(Sen3)))
print("Spc: %.2f%% (+/- %.2f%%)" % (np.mean(Spc3), np.std(Spc3)))
print("AUC: %.2f%% (+/- %.2f%%)" % (np.mean(AUC3), np.std(AUC3)))
print("Training Time: %.4f s" % (np.mean(Train_time3)))
print("Testing Time: %.4f s" % (np.mean(Test_time3)))
print("-------------DTCNN_SVM-------------")
print("-----------------------------------")